In [1]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import tensorflow as tf
from glob import glob
from yolov3.yolov4_config import Create_Yolo
from yolov3.utils_config import load_yolo_weights, detect_image
from yolov3.configs_config import *
from PIL import Image

import os
import pandas as pd

if YOLO_TYPE == "yolov4":
    Darknet_weights = YOLO_V4_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V4_WEIGHTS
if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS

yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE)
load_yolo_weights(yolo, Darknet_weights) # use Darknet weights

In [2]:
yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
yolo.load_weights("./checkpoints/frist/yolov3_custom") # use keras weights

In [3]:
r1 = glob(r"teeth_including_lip_more\test/*.jpg")

In [4]:
r11 = glob(r"teeth_including_lip_more\train/*.jpg")

In [5]:
def GIoU(bboxes_1, bboxes_2):
    # 1. calulate intersection over union
    area_1 = (bboxes_1[2] - bboxes_1[0]) * (bboxes_1[3] - bboxes_1[1])
    area_2 = (bboxes_2[2] - bboxes_2[0]) * (bboxes_2[3] - bboxes_2[1])
    
    intersection_wh = tf.minimum(bboxes_1[2:], bboxes_2[2:]) - tf.maximum(bboxes_1[ :2], bboxes_2[ :2])
    intersection_wh = tf.maximum(intersection_wh, 0)
    
    intersection = intersection_wh[0] * intersection_wh[1]
    union = (area_1 + area_2) - intersection
    
    ious = intersection / union
    
    # 2. (C - (A U B))/C
    C_wh = tf.maximum(bboxes_1[2:], bboxes_2[2:]) - tf.minimum(bboxes_1[:2], bboxes_2[:2])
    C_wh = C_wh
    C = C_wh[0] * C_wh[1]

    giou = ious - (C - union) / C
    return giou, ious

In [6]:
file = open("model_data/lip_more_train.txt", "r")
no_lip_train = file.readlines()
file.close()
file = open("model_data/lip_more_validation.txt", "r")
no_lip_valid = file.readlines()
file.close()

In [7]:
GT_train = []
GT_valid = []

In [8]:
giou_ious_list_train = []
giou_ious_list_valid = []

In [9]:
for i in range(len(r11)):
    bb = no_lip_train[i].split()[1].split(",")[0:4]
    bb = [int(item) for item in bb]
    GT_train.append(bb)
for i in range(len(r1)):
    bb = no_lip_valid[i].split()[1].split(",")[0:4]
    bb = [int(item) for item in bb]
    GT_valid.append(bb)

In [10]:
# train
for i in range(len(r11)):
    image, bboxes = detect_image(yolo, r11[i], "", input_size=YOLO_INPUT_SIZE, show=False, score_threshold=0.1, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    if len(bboxes) == 0 :
        giou_ious_list_train.append([0,0])
    else :
        giou,ious = GIoU(np.array(GT_train[i]), bboxes[0][:4])
        giou_ious_list_train.append([giou,ious])

In [11]:
# validation
for i in range(len(r1)):
    image, bboxes = detect_image(yolo, r1[i], "", input_size=YOLO_INPUT_SIZE,score_threshold=0.1, show=False, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    if len(bboxes) == 0 :
        giou_ious_list_valid.append([0,0])
    else :
        giou,ious = GIoU(np.array(GT_valid[i]), bboxes[0][:4])
        giou_ious_list_valid.append([giou,ious])

In [12]:
train_names = []
test_names = []
for i in range(len(r11)):
    train_names.append(os.path.basename(r11[i]))
for i in range(len(r1)):
    test_names.append(os.path.basename(r1[i]))

In [13]:
raw_training = pd.DataFrame(data=np.array(giou_ious_list_train), index=[train_names], columns=["giou", "iou"])

In [14]:
raw_test = pd.DataFrame(data=np.array(giou_ious_list_valid), index=[test_names], columns=["giou", "iou"])

In [15]:
xlxs_dir='no_prob_lip_giou.xlsx' #경로 및 파일명 설정
with pd.ExcelWriter(xlxs_dir) as writer:
    raw_training.to_excel(writer, sheet_name = 'training_data') #raw_data1 시트에 저장
    raw_test.to_excel(writer, sheet_name = 'test_data') #raw_data2 시트에 저장

In [16]:
giou_ious_list_train

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.9820678346961164>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.9806049297849356>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.9458796804651597>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.9442113831319267>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.9613514823217222>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.9609191514946546>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.9882296199982199>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.9914229886624406>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.9418873073815573>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.9449779307445746>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.9844192417748068>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.9830183091831838>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.9789293955435294>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.981090939583246>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.974370470510729>,
  <tf